In [ ]:
# === 1. Setup ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use('seaborn')



In [ ]:
# === 2. Load VIX, VIX3M and SKEW ===
import pandas as pd, pathlib, urllib.request

root = pathlib.Path('data/vol')
root.mkdir(parents=True, exist_ok=True)

urls = {
    'VIX'   : 'https://stooq.com/q/d/l/?s=%5Evix&i=d',
    'VIX3M' : 'https://stooq.com/q/d/l/?s=%5Evix3m&i=d',
    'SKEW'  : 'https://stooq.com/q/d/l/?s=%5Evskt&i=d'
}

dfs = []
for name, url in urls.items():
    csv_path = root / f'{name}.csv'
    if not csv_path.exists():
        urllib.request.urlretrieve(url, csv_path)
    df = pd.read_csv(csv_path, parse_dates=['Date'])
    df = df[['Date','Close']].rename(columns={'Close': name})
    dfs.append(df)

vol = dfs[0]
for d in dfs[1:]:
    vol = vol.merge(d, on='Date', how='inner')

vol.set_index('Date', inplace=True)
vol.tail()


In [ ]:
# === 2b. Pull SPY prices ===
spy = yf.download('SPY', start=vol.index.min(), end=vol.index.max())['Adj Close']
spy = spy.rename('SPY')



In [ ]:
# === 3. Build signals ===
# Term‑structure slope (90d minus 30d)
vol['slope'] = vol['VIX3M'] - vol['VIX']

# Skew percentile (higher SKEW = pricey crash protection)
vol['skew_pct'] = vol['SKEW'].rolling(252).rank(pct=True)  # 1‑year look‑back

# Monthly sample: pick last trading day of each month
signals = vol[['slope','skew_pct']].resample('M').last()
spy_m   = spy.resample('M').last()


In [ ]:
# === 3. Trading rule and backtest ===

# Long SPY if term structure is steep (slope > 0) AND skew is cheap (<50th pctile)
long_signal  = (signals['slope'] > 0) & (signals['skew_pct'] < 0.5)

# Compute next‑month returns
spy_ret = spy_m.pct_change().shift(-1)   # forward return
strategy_ret = spy_ret.where(long_signal, -spy_ret)   # short when rule is false

perf = pd.DataFrame({'Strategy': strategy_ret, 'SPY': spy_ret})
perf.dropna(inplace=True)

# Metrics
sharpe = perf['Strategy'].mean() / perf['Strategy'].std() * np.sqrt(12)
cagr   = (1 + perf['Strategy']).prod()**(12/len(perf)) - 1
maxdd  = (perf['Strategy'].cumsum() - perf['Strategy'].cumsum().cummax()).min()

print(f"Sharpe {sharpe:.2f} | CAGR {cagr:.2%} | MaxDD {maxdd:.2%}")


In [ ]:
# === 5. Plot ===
cum = (1 + perf).cumprod()
cum.plot(figsize=(8,4), title='Cumulative Return: Strategy vs SPY')
plt.xlabel('')
plt.show()



In [ ]:
# === 6. Analyze Performance ===
# (To be implemented next)
